# Set relevant variables

In [1]:
path_to_aspera_licence = '/data/pacbio_assembly/pb_data/asperaweb_id_dsa.openssh'

# Import libraries

In [2]:
import os

# Download the relevant files

We use [aspera](http://www.asperasoft.com/downloads/connect) to download. Other ways to download the relevant files from the accession numbers in the paper can be found [here](https://www.ebi.ac.uk/ena/browse/read-download).

In [3]:
download_cmd = 'python download_wrapper.py '+ path_to_aspera_licence
os.system(download_cmd)